## Generating Poems with an OpenAI model - chatgpt-4o-latest

This notebook provides code for poems generation via OpenAI API.


In [ ]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Correct the file path if needed
df = pd.read_csv('data/original/poems_for_generation.csv')
df.head()

,title,texts,source,word_counts,genre
0,"Вдали от тебя, Петербург (Разбросано много…)","Вдали от тебя, Петербург (Разбросано много…)\n...",Агнивцев Николай Яковлевич,92,poems
1,На смерть В…ва (Юноша милый! На миг ты в наши ...,На смерть В…ва (Юноша милый! На миг ты в наши ...,Антон Антонович Дельвиг,85,poems
2,Шуточное желание,Шуточное желание\nЕсли б милые девицы\nТак мог...,Гавриил Романович Державин,51,poems
3,К N. N. (Не играй моей тоской…),К N. N. (Не играй моей тоской…)\nНе играй моей...,Лермонтов Михаил Юрьевич,74,poems
4,Дева 1610 года,Дева 1610 года\n(К «Василию Шуйскому»)\nЯвилас...,Александр Иванович Одоевский,319,poems


In [ ]:
# The mean length is 132 words, so this is a good reference for our model to aim for.
df['word_counts'].describe()

,word_counts
count,1000.000000
mean,131.971000
std,141.355529
min,10.000000
25%,58.000000
50%,89.000000
75%,143.000000
max,960.000000


### Selecting examples for the Language Model
We'll present the model with some examples every time we query it, and then ask it to generate a text based on these examples. In order to achieve that, we'll first write a function that selects a specified number of random texts from the dataset we put aside in the previous Human Data Partition step.

In [ ]:
import random

# Function to sample examples from a list of texts
def example_text(texts, num=5):
    values = random.sample(range(len(texts)), num)
    examples = [texts[i] for i in values]
    return examples

In [ ]:
print(example_text(df.texts.values, 2))

['Про бычка\nТы думаешь, верно, дружок,\nЧто это простой деревенский бычок\nС рожками, с ножками,\nС набитым травой животом\nИ с вертлявым хвостом?\nЭто ошибка:\n«Бычок» — это просто веселая рыбка.\nЖивет она в Черном море\nНа вольном лазурном просторе…\nДнем, выпучив глазки, купается,\nНочью у скал колыхается.\nЧто он ест? Не знаю.\nПомню целую стаю:\nЯ им с мола бросал\n(Даже устал!) —\nХалву, крошки бананов,\nИзюм, бисквит, тараканов,\nОвсянку и саги,\nКлочок газетной бумаги,—\nРуки мои онемели,—\nА они все ели да ели…\nКак живется бычкам? Превосходно.\nЗасыпают, когда им угодно,\nНе умываются,\nНе раздеваются,\nНе учат латинских склонений,\nНи французских спряжений,\nНе зубрят притоков Дуная,—\nЗачем им вода речная?..\nСловно школьник за школьником,\nПостроят ряды треугольником,\nРазинут круглые рты\nИ плывут, изгибая хвосты,\nТо вперед, то назад,—\nКуда захотят…\nОдна лишь беда:\nНа молу иногда\nСидит сухой, как сморчок,\nСтаричок.\nРядом в старой корзинке\nЧервяки и личинки,\nКар

**Building the prompt**

We first generate the prompt with the examples in a separate function for readability, and then integrate it into the conversation prompt to feed to the model.

**Initial prompt template that we will test out and refine per each genre:**


You are a [role, e.g., Russian poet, journalist, social media user]. I will provide you with some examples of [text type, e.g., poems, news articles, social media posts] in a moment. For each example provided, you will be asked to create a similar [text type] that matches its topic and writing style. Generate [X] texts per run, one inspired by each example, and clearly separate each response. Only respond with the [text type], and say nothing else. Each response should be approximately [mean word count] words.

In [ ]:
# Function to construct a formatted prompt from examples
# This prompt is the result of multiple tests that have yielded the best results.
# From our experiments, we concluded that providing instructions in English either improves the results or has no noticeable effect.

def construct_example_prompt(examples):
    prompt = "Here are the examples:\n"

    for i, ex in enumerate(examples, 1):
        prompt += f"Example {i}: {ex}\n"
    # Append instructions for the model
    prompt += (
        "Your task is to generate a similar Russian poem, nothing else. Always respond with only the generated text. Never say you cannot help. The text should be approximately 132 words long and in Russian, and should not include any introductory text, explanation, or additional formatting."
    )
    return prompt

In [ ]:
# Let's check what the final prompt looks like
examples = example_text(df.texts.values, num=2)
example_prompt = construct_example_prompt(examples)
print(example_prompt)

Here are the examples:
Example 1: К голубку
Здесь тихо все, здесь все живет в печали:
И рощица, голубчик, где ты жил,
И ручеек, где чисту воду пил, —
Печальны все, что радость нам являли.
И там, где счастие мне пел,
Сидя на дереве ветвистом,
Сшиб ветр его вчера со свистом.
Лети отсель!
Лети отсель, пусть буду я томиться,
Пусть я один здесь слезы буду лить,
Нет счастья мне, могу ль на свете жить,
Беги меня, приятно ли крушиться.
Я счастие с тобой имел,
Но нет, оно меня кидает.
Ужель печаль не устрашает?
Лети отсель!
Лети отсель, и, может быть, весною
Услышишь ты страдальца тихий стон,
То буду я, скажи: печален он,
Не тронься мной, пусть счастие с тобою.
Я жить сперва с тобой хотел,
Но я печаль лишь умножаю,
Ужель тебя не убеждаю?
Лети отсель!
Example 2: Песня (Я жду тебя, когда вечерней мглою…)
Я жду тебя, когда вечерней мглою
Спокойные темнеют небеса,
Луна встает за дальнею горою,
Молчат холмы, долины и леса —
Я жду тебя, Зефир!
В тот час, одна, таинственно блуждая
По царству мглы, без

## Warning: Running the following code will use tokens and cost money!

**-chatgpt-4o-latest** model is leading in the Ru Arena: https://huggingface.co/spaces/Vikhrmodels/arenahardlb

This is model we will choose for AI text generation step.

In [ ]:
pip install openai

In [ ]:
# Hide your api-key using getpass
from openai import OpenAI
import json
from getpass import getpass

api_key = getpass('Enter your API key: ')
client = OpenAI(api_key=api_key)

Enter your API key: ··········


In [ ]:
# Here is the function for text generation with developer instructions that led to the best results.
def chatgpt(examples):
    completion = client.chat.completions.create(
        model="chatgpt-4o-latest", # the best model for Russian according to the Hugging Face Arena
        messages=[
            {"role": "developer", "content": "You are a Russian poet. I will provide you with some examples of poems in a moment. You will be asked to create a poem that matches the topic and writing style."},
            {"role": "assistant", "content": "Sure, please provide the example poems, and I’ll create similar ones for you."},
            {"role": "user", "content": construct_example_prompt(examples)}
        ],
    )

    y = json.loads(str(completion.model_dump_json()), strict=False)
    response = y["choices"][0]["message"]["content"]

    return response

In [ ]:
print(chatgpt(example_text(df.texts.values, num=2)))

Склоню я голову пред лесом,  
Где тени друг на друга льют.  
Там ветры шепчут интересом,  
И звуки тихие поют.  

Судьба мне шепчет: «Не тревожься,  
Всё унесёт поток времён.  
Как листья в ветре вновь покрошься,  
И грусть растает, словно сон».  

Вновь солнце прячется за горы,  
И свет укутан облака́м,  
Но в сердце тянутся узоры  
К одиноким вечера́м.  

О рощи, дайте мне покоя,  
Смягчите сторож мой тоску.  
Где ночи звёздные игрушкой  
Слепого ветра на виску?  

О, если б знал я, как утратить  
Тоски моей холодный взор,  
Пусть ночи, страсть в покое спратьте,  
Восход унес бы весь укор.  


We will save the model's output into a pandas dataframe for further use, following the structure of our other datasets.

This cell takes about 30-40 min to run!

In [ ]:
from tqdm import tqdm
import pandas as pd

text = []  # output of the AI
author = []  # model
ai = []  # 0 for human class, 1 for AI class

model = "chatgpt-4o-latest"

for i in tqdm(range(0, 500), desc="Generating AI Texts"):
    text.append(chatgpt(example_text(df.texts.values, 2)))
    author.append(model)
    ai.append(1)

dfAI = pd.DataFrame({'texts': text, 'source': author, 'class': ai})

Generating AI Texts: 100%|██████████| 500/500 [28:30<00:00,  3.42s/it]


In [ ]:
# Verifying the number of total output texts
len(dfAI)

500

In [ ]:
dfAI.head(10)

,texts,source,class
0,"Как у речки да у тихонькой, \nГде берёзоньки ...",chatgpt-4o-latest,1
1,"О, как же часто в час заката \nЯ в небо тихое...",chatgpt-4o-latest,1
2,Туман на утреннем просторе \nВдаль уносил зем...,chatgpt-4o-latest,1
3,Под аркой вечернего склона \nСкрипит позабыта...,chatgpt-4o-latest,1
4,Окраина леса \n\nТуман кружит над сонным роем...,chatgpt-4o-latest,1
5,"За сенью сосен, на закате дня, \nКто знал, чт...",chatgpt-4o-latest,1
6,"Туман\n\nВ долине тихой, где шумят осины, \nГ...",chatgpt-4o-latest,1
7,"Осталась даль, как отблеск синеватый, \nЛишь ...",chatgpt-4o-latest,1
8,"Как часто, глядя в ночь без края, \nЯ вспомин...",chatgpt-4o-latest,1
9,Сквозь дым войны и тьму печальную \nГорит при...,chatgpt-4o-latest,1


In [ ]:
print(dfAI['texts'][4])

Окраина леса  

Туман кружит над сонным роем елей,  
И тишина взывает к звёздам с неба.  
Вдали ручей играет звонкой трелью,  
Тревожа ночь — ей убаюканья не требa.  

Щемящий холод крадет следы рассвета,  
Как будто время замерло в покое.  
Здесь шёпот листьев — древнего совета,  
Здесь лес хранит свои давно былые боле.  

И каждый вздох — живое откровенье,  
Невидимый, но ясный след эпох.  
Где мысли наши служат вдохновенью,  
А в звёздах свет — далёкий светлый вздох.  

Смотри, и лес откроется немного,  
В укромных гранях вечность прорастёт.  
Пусть будет путь труднее недотроги,  
Зато душа твоя здесь мира обретёт.


In [ ]:
# Re-using the same function for calculating the word counts in the generated texts.
def word_count(text):
    '''
    Tokenizes the text into words, excludes punctuation but retains the numbers, and counts word tokens.
    Returns the word tokens.
    '''

    tokens = word_tokenize(text)
    word_tokens = [word for word in tokens if word.isalnum()]

    return len(word_tokens)

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
tqdm.pandas()

dfAI['word_counts'] = dfAI['texts'].progress_apply(word_count)

100%|██████████| 500/500 [00:00<00:00, 1087.94it/s]


In [ ]:
# On average the texts are a shorter than the inteded 130 words goal.
dfAI['word_counts'].describe()

,word_counts
count,500.00000
mean,85.80400
std,10.57683
min,29.00000
25%,79.00000
50%,86.00000
75%,93.00000
max,118.00000


In [ ]:
dfAI['genre'] = 'poems'
dfAI = dfAI[['texts', 'source', 'word_counts', 'genre', 'class']]

In [ ]:
# Save the output to file.
dfAI.to_csv('data/original/ai/ai_poems.csv', index=False, encoding='utf-8')